# Custom RAG Pipeline w/ History Implementation

This program will be adding history to our RAG pipeline, modifying history from built in RAG history retrievers to creating custom chains with the "|" operator. Creating a custom retriever allows us better control over context injection, injecting in smaller chunks with metadata chunks.

In [1]:
import os
import faiss
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM

In [2]:
MODEL_NAME = "llama3.2"
llm = OllamaLLM(model= MODEL_NAME)

In [3]:
def load_docs(pdf_folder = "./pdf_folder"):
    
    document_loader = []

    for root, dirs, files in os.walk(pdf_folder):
        for file in files:
            if file.lower().endswith(".pdf"):
                full_path = os.path.join(root, file)
                document_loader.append(full_path)

    return document_loader

In [4]:
document_loader = load_docs()
document_loader

['./pdf_folder/ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf',
 './pdf_folder/Three Phase Power System Fundamentals.pdf',
 './pdf_folder/ENSC3016_Course_Notes_Part_2_Electric_Machines.pdf',
 './pdf_folder/Electric Machinery Fundamentals Textbook -- Chapman.pdf',
 './pdf_folder/ENSC3016 Study Guide 1-Review of Circuit Fundamentals.pdf']

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
model.save("./local_models/all-MiniLM-L6-v2")

/home/Akshay/miniconda3/envs/LLM_Train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embedding_model ="./local_models/all-MiniLM-L6-v2" #embedding matrix model

def embed_splitting(document_loader, embedding_model):
    embeddings = HuggingFaceEmbeddings(model = embedding_model, encode_kwargs={'normalize_embeddings': True})

    doc_store = []
    for file_path in document_loader:
        loader = PyPDFLoader(file_path)
        docs = loader.load()

        # Clean the metadata: keep only the filename, not full path
        for doc in docs:
            doc.metadata["source"] = os.path.basename(file_path)

        doc_store += docs


    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size = 400,
        chunk_overlap = 64
        )
    
    #Make splits
    splits = text_splitter.split_documents(doc_store)

    return embeddings, splits

In [6]:
import os
import pickle

SPLITS_CACHE_PATH = "splits_cache.pkl"

def get_splits(document_loader, embedding_model):
    if os.path.exists(SPLITS_CACHE_PATH):
        print("Loading cached splits from disk...")
        with open(SPLITS_CACHE_PATH, "rb") as f:
            splits = pickle.load(f)
        embeddings = HuggingFaceEmbeddings(model=embedding_model, encode_kwargs={'normalize_embeddings': True})
    else:
        print("Creating new splits...")
        embeddings, splits = embed_splitting(document_loader, embedding_model)
        with open(SPLITS_CACHE_PATH, "wb") as f:
            pickle.dump(splits, f)
    return embeddings, splits

In [7]:
embeddings, splits = get_splits(document_loader, embedding_model)

Loading cached splits from disk...


/home/Akshay/miniconda3/envs/LLM_Train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
example_split = splits[106]
example_split

Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}, page_content='Transformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe action of a transformer is most easily underst

In [9]:
metadata = example_split.metadata
for key in metadata:
    print(f"{key}: {metadata[key]}")

producer: Microsoft® Word 2013
creator: Microsoft® Word 2013
creationdate: 2019-07-27T15:04:48+08:00
author: Ali Kharrazi
moddate: 2019-07-27T15:04:48+08:00
source: ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf
total_pages: 76
page: 51
page_label: 52


In [10]:
embeddings

HuggingFaceEmbeddings(model_name='./local_models/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={'normalize_embeddings': True}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
len(splits)

402

In [12]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [13]:
dim = len(embeddings.embed_query("test sentence"))
index = faiss.IndexFlatL2(dim)

if os.path.exists("faiss_index"):
    print("Loading FAISS index from disk...")
    vector_store = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)
else:
    print("Building FAISS index from scratch...")
    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    vector_store.add_documents(splits)
    vector_store.save_local("faiss_index")

Loading FAISS index from disk...


In [14]:
# create the retriever object once
semantic_retriever = vector_store.as_retriever(search_kwargs={'k': 4})

# define your function to query it
def semantic_search(retriever_obj, input_context: str):
    return retriever_obj.invoke(input_context)

# call the function with retriever and query string
results = semantic_search(semantic_retriever, "Explain transformers")

In [15]:
results[0].metadata

{'producer': 'Microsoft® Word 2013',
 'creator': 'Microsoft® Word 2013',
 'creationdate': '2019-07-27T15:04:48+08:00',
 'author': 'Ali Kharrazi',
 'moddate': '2019-07-27T15:04:48+08:00',
 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf',
 'total_pages': 76,
 'page': 51,
 'page_label': '52'}

In [16]:
for i in range(len(results)):
    source_data = results[i].metadata["source"]
    page = results[i].metadata["page"]
    page_content = results[i].page_content

    print(f"This is chunk number {i+1}.\n\n The source is {source_data}, found on page number {page}. \n\n The page content is {page_content} \n")

This is chunk number 1.

 The source is ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, found on page number 51. 

 The page content is Transformer 52 
 
 
 
   Figure 6-3 Shell-type transformers. 
 
 
 
Figure 6-4 Flux plot: shell-type transformer 
 
 
Toroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. 
Although they are more expensive than shell-type transformers, the performance is better. They are used 
in high -quality electronic equipment and for instrument transformers (see section 6.3) where 
measurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. 
 
Figure 6-5 Toroidal transformers.
 
 
 
6.2 Transformer Principle: 
The action of a transformer is most easily understood if the two coils are wound on opposite sides of a 
magnetic core, as shown in the model of figure 6 -6. This form is used for some low -cost transformers, 
but the magnetic coupling is not as good as with the shel

In [17]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 4

def keyword_search(retriever_obj, input_context: str):
    return retriever_obj.invoke(input_context)

In [21]:
keyword_results = keyword_search(bm25_retriever, "Explain transformers")

In [22]:
keyword_results 

[Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 64, 'page_label': '65'}, page_content='65 Electrical Machines and Systems                                                                                                            \n6.8 Current Transformers \nInstrument transformers are special transformers for extending the range of measur ing instruments. \nThere are two basic types: voltage transformers for measuring high voltages, and current transformers \nfor measuring high currents. Using transformers for voltage measurement is similar in principle to the \nordinary use of transformers to ch ange voltage levels, so it will not be considered further. Current \ntransformers, on the other hand, need special consideration. These are

In [23]:
for i in range(len(keyword_results)):
    source_data = keyword_results[i].metadata["source"]
    page = keyword_results[i].metadata["page"]
    page_content = keyword_results[i].page_content

    print(f"This is chunk number {i+1}.\n\n The source is {source_data}, found on page number {page}. \n\n The page content is {page_content} \n")

This is chunk number 1.

 The source is ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, found on page number 64. 

 The page content is 65 Electrical Machines and Systems                                                                                                            
6.8 Current Transformers 
Instrument transformers are special transformers for extending the range of measur ing instruments. 
There are two basic types: voltage transformers for measuring high voltages, and current transformers 
for measuring high currents. Using transformers for voltage measurement is similar in principle to the 
ordinary use of transformers to ch ange voltage levels, so it will not be considered further. Current 
transformers, on the other hand, need special consideration. These are usually toroidal transformers with 
high-quality core material. 
Figure 6-25 shows a load connected to a source. The primary of a current transformer is in series with 
the load, and the secondary 

In [24]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(retrievers= [semantic_retriever, bm25_retriever], weights = [0.5, 0.5])

In [25]:
combined_results = ensemble_retriever.invoke("Explain transformers")

In [26]:
combined_results

[Document(id='cf3a2be7-1574-4c4a-94f0-6c84e36a3a2e', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}, page_content='Transformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe ac

In [27]:
for i in combined_results:

    print(i.metadata)

{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 64, 'page_label': '65'}
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 50, 'page_label': '51'}
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013'

In [28]:
i = 1
result_list = [combined_results[0]]
seen_pages = {combined_results[0].metadata["page_label"]}

while i < len(combined_results):
    metadata = combined_results[i].metadata
    page_label = metadata["page_label"]

    if page_label in seen_pages:
        i += 1  # You MUST increment i here
        continue

    result_list.append(combined_results[i])
    seen_pages.add(page_label)
    i += 1

In [29]:
result_list

[Document(id='cf3a2be7-1574-4c4a-94f0-6c84e36a3a2e', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}, page_content='Transformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe ac

In [30]:
input_template = """You are an expert assistant answering based only on the provided context.

The retrieved documents have been joined together and are separated by "Chunk_n", where n is the chunk number. Here is the context:

{context}

Use **ALL** relevant information above to answer the question below. If the answer isn't found in the chunks, say:
"I cannot answer this question because the necessary information was not found in the provided documents."

❗Do not cite or mention any source files or page numbers in the body of your answer.

At the end of your answer, add a single line in this format:

Information was pulled from: <source_file_1>: pages <comma-separated page numbers>; <source_file_2>: pages <...>; ...

Use only one entry per document, listing all unique page numbers where information was pulled from.
Do not mention metadata_n, chunk_n, or include references in the main answer.

Metadata:
{metadata}

Question: {question}
"""

In [31]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnableParallel

def content_parser(results):
    context_string = ""
    for i in range(len(results)):

        context_string += f"\nChunk_{i+1}:\n\n{results[i].page_content}\n\n\n"

    return context_string 

chunks = content_parser(result_list)

In [32]:
chunk_runnable = RunnableLambda(content_parser)

In [33]:
def metadata_parser(results):
    metadata_files = {}

    for doc in results:
        source = doc.metadata["source"]
        page = doc.metadata["page_label"]

        if source in metadata_files:
            if page not in metadata_files[source]:
                metadata_files[source].append(page)
        else:
            metadata_files[source] = [page]

    metadata_string = "This file uses the following sources:"

    for key in metadata_files:
        pages = ", ".join(str(i) for i in metadata_files[key])
        metadata_string += f"\n\n{key}, pages {pages}"
        

    return metadata_string

In [34]:
metadata_runnable = RunnableLambda(metadata_parser)
print(metadata_runnable.invoke(result_list))

This file uses the following sources:

ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, pages 52, 65, 51, 53, 56, 69


In [35]:
metadata = metadata_parser(result_list)

In [36]:
print(metadata)

This file uses the following sources:

ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, pages 52, 65, 51, 53, 56, 69


In [37]:
from embed_splitting import load_docs, get_splits

In [38]:
prompt_template = ChatPromptTemplate(
    [
        ("system", input_template),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt_template | llm

In [39]:
contextualize_q_system_prompt = """
You are a question reformulator in a retrieval-based QA system.

Given the latest user question and the preceding chat history, your task is:

1. If the question is fully self-contained — i.e., it is grammatically and semantically complete and understandable on its own — return it **exactly as-is**.

2. If the question is ambiguous without the chat history or depends on previous turns, rewrite it into a fully standalone, self-contained question.

⚠️ Do NOT answer the question.  
⚠️ Do NOT add any preamble, commentary, or extra explanation.  
⚠️ Output **only** the final question text (either original or reformulated).

Your job is to produce a single, context-independent question if needed — nothing else.
"""

contextual_prompt = ChatPromptTemplate(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [40]:
answer1 = chain.invoke(
    {'context': chunks,
     'metadata': metadata,
     'chat_history': [],
     'input': "Explain transformers",
     'question': "Explain transformers"}
)

In [41]:
answer1

'Transformers are a practical application of magnetically coupled coils that transfer energy from one coil to another. They are essentially AC devices used for electrical isolation and voltage transformation. The coils are usually placed on a common magnetic core to improve the coupling.\n\nIn a transformer, the primary coil is connected to the source, and the secondary coil is connected to the load. When an alternating current flows through the primary coil, it creates a changing magnetic flux that induces a voltage in the secondary coil. This process allows transformers to change the voltage and current levels of electrical signals.\n\nThe core material used in high-frequency transformers is often magnetically soft ferrites, while power frequency transformers use iron alloys such as silicon steel. The cores are designed to improve the coupling between the primary and secondary coils.\n\nTransformers can be used for various purposes, including step-down and step-up transformations, vo

In [42]:
a = RunnableParallel({'context': chunk_runnable, 'metadata': metadata_runnable})
b = a.invoke(result_list)

In [43]:
b

{'context': '\nChunk_1:\n\nTransformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe action of a transformer is most easily understood if the two coils are wound on opposite sides of a \nmagnetic core, as shown in the model of figure 6 -6. This form is used for some low -cost transformers, \nbut the magnetic coupling is not as good as with the shell-type construction. \n \n \nFigure 6-6  Core-type transformer \n \n \n \nFigure 6 -7 is a schema

In [44]:
inputs = ({
    **b,
    'chat_history': [],
    'input': 'Explain transformers',
    'question': 'Explain transformers'
})

In [45]:
inputs

{'context': '\nChunk_1:\n\nTransformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe action of a transformer is most easily understood if the two coils are wound on opposite sides of a \nmagnetic core, as shown in the model of figure 6 -6. This form is used for some low -cost transformers, \nbut the magnetic coupling is not as good as with the shell-type construction. \n \n \nFigure 6-6  Core-type transformer \n \n \n \nFigure 6 -7 is a schema

In [46]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [55]:
answers = ensemble_retriever.invoke("Explain transformers")

metadata_dict = {}

for i in range(len(answers)):
    metadata_dict[f"metadata {i}"] = answers[i].metadata

answers[0].metadata

{'producer': 'Microsoft® Word 2013',
 'creator': 'Microsoft® Word 2013',
 'creationdate': '2019-07-27T15:04:48+08:00',
 'author': 'Ali Kharrazi',
 'moddate': '2019-07-27T15:04:48+08:00',
 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf',
 'total_pages': 76,
 'page': 51,
 'page_label': '52'}

In [47]:
history_aware_chain = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [49]:
get_session_history('abb73283').messages

[]

In [54]:
rephrase_pipe

In [73]:
import uuid

def temp_pipeline():

    session_id = str(uuid.uuid4())[:8]
    print(f"Session ID: {session_id}")
    
    history = get_session_history(session_id)

    print(f"\nModel {MODEL_NAME} has been initiated with memory. Please feel free to ask questions or type 'exit' to quit.")
    while True:
        
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Session ended. Have a good day.")
            break

        print(f"{user_input}\n\n\n")
        
        recontextual_chain = contextual_prompt | llm
        rephrased_question = recontextual_chain.invoke(
            {'chat_history': history.messages,
             'input': user_input})
        
        print(f"{rephrased_question} \n\n\n")

        context_injection = (ensemble_retriever | RunnableParallel({'context': chunk_runnable, 'metadata': metadata_runnable})).invoke(rephrased_question)

        print("Metadata:\n", context_injection['metadata'])
        
        response = history_aware_chain.invoke(
            {**context_injection,
            'input': user_input,
            'question': rephrased_question},
            config={"configurable": {"session_id": session_id}}
        )
        
        print(f"LLM: {response}\n")

In [74]:
temp_pipeline()

Session ID: a797f7a3

Model llama3.2 has been initiated with memory. Please feel free to ask questions or type 'exit' to quit.
How do trandsforemers wprk?



How do transformers work? 



Metadata:
 This file uses the following sources:

ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, pages 52, 51, 65, 69, 20

ENSC3016_Course_Notes_Part_2_Electric_Machines.pdf, pages 60
LLM: Transformers are essentially AC devices that transfer energy from one coil to another through magnetically coupled coils. The coils are usually placed on a common magnetic core to improve the coupling. A transformer consists of two coils, the primary and secondary, connected by a magnetic core.

The action of a transformer is most easily understood if the two coils are wound on opposite sides of a magnetic core, as shown in the model of figure 6-6. The coil connected to the source is termed the primary, and the coil connected to the load is termed the secondary.

When an alternating current flows th

In [66]:
import uuid

history1 = []

In [81]:

def pipeline():

    session_id = str(uuid.uuid4())[:8]
    print(f"Session ID: {session_id}")
    
    history = get_session_history(session_id)

    print(f"\nModel {MODEL_NAME} has been initiated with memory. Please feel free to ask questions or type 'exit' to quit.")
    while True:
        
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Session ended. Have a good day.")
            break

        print(f"{user_input}\n\n\n")
        
        MAX_HISTORY_TURNS = 1
        recontextual_chain = contextual_prompt | llm
        rephrased_question = recontextual_chain.invoke(
            {'chat_history': history.messages[-MAX_HISTORY_TURNS:],
             'input': user_input})
        
        print(f"{rephrased_question} \n\n\n")

        context_injection = (ensemble_retriever | RunnableParallel({'context': chunk_runnable, 'metadata': metadata_runnable})).invoke(rephrased_question)

        expected_context = ensemble_retriever.invoke(user_input)
        rephrased_context = ensemble_retriever.invoke(rephrased_question)

        for i in expected_context:
            source = i.metadata["source"]
            page_label = i.metadata["page_label"]
            print(f"Expected metdata is:\n\n {source}, page number {page_label}")

        for i in rephrased_context:
            source = i.metadata["source"]
            page_label = i.metadata["page_label"]
            print(f"Rephrased question metdata is:\n\n {source}, page number {page_label}")
        
        print(f"Metadata:\n, {context_injection['metadata']}\n\n")
        
        response = history_aware_chain.invoke(
            {**context_injection,
            'input': user_input,
            'question': rephrased_question},
            config={"configurable": {"session_id": session_id}}
        )
        
        print(f"LLM: {response}\n")


In [82]:
pipeline()

Session ID: 8556bd5e

Model llama3.2 has been initiated with memory. Please feel free to ask questions or type 'exit' to quit.
How do transformers work?



What are transformers? 



Expected metdata is:

 ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, page number 52
Expected metdata is:

 ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, page number 51
Expected metdata is:

 ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, page number 65
Expected metdata is:

 ENSC3016_Course_Notes_Part_2_Electric_Machines.pdf, page number 60
Expected metdata is:

 ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, page number 51
Expected metdata is:

 ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, page number 69
Expected metdata is:

 ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, page number 20
Rephrased question metdata is:

 ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, page number 51
Rep

In [1]:
get_session_history('37b5e65a').messages

NameError: name 'get_session_history' is not defined